In [1]:
from FINDER import FINDER
import os
import numpy as np
import pandas as pd
import tsplib95
import networkx as nx


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
dqn = FINDER()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
data_dir = 'test_sets/synthetic_nrange_10_20_1000/'

In [4]:
import re
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

fnames = os.listdir(data_dir)
fnames.sort(key=natural_keys)

In [5]:
graph_list = []
fnames = os.listdir(data_dir)
fnames.sort(key=natural_keys)
for fname in fnames:
    try:
        problem = tsplib95.load(data_dir + fname)
        graph_list.append(problem.get_graph())
    except:
        print('Error, while loading file {}'.format(fname))

In [6]:
# remove edges from one node to itself
for g in graph_list:
    ebunch=[(k,k) for k in range(len(g.nodes))]
    g.remove_edges_from(ebunch)

In [7]:
# scale size of the graphs such that it fits into 0,1 square
scale_factor = 1/10000
for g in graph_list:
    for node in range(len(g.nodes)):
        g.nodes[node]['coord'] = np.array(g.nodes[node]['coord']) * scale_factor
    for edge in g.edges:
        g.edges[edge]['weight'] = g.edges[edge]['weight'] * scale_factor

In [35]:
best_model = dqn.LoadbestModel()
best_model

INFO:tensorflow:Restoring parameters from best_models/tsp_2d/nrange_10_20_len_025792/nrange_10_20_iter_56100_len_025792.ckpt
model sucessfully restored from file


'nrange_10_20_iter_56100_len_025792'

In [10]:
# load model into Finder
g_type = dqn.g_type
best_model = dqn.LoadbestModel()
lengths = []
solutions = []
for g in graph_list:
    len, sol, time = dqn.Evaluate(g)
    lengths.append(len)
    solutions.append(sol)


INFO:tensorflow:Restoring parameters from best_models/tsp_2d/nrange_10_20_len_025792/nrange_10_20_iter_56100_len_025792.ckpt
model sucessfully restored from file


In [11]:
np.mean(lengths)

0.25953067493598037

In [14]:
index = pd.RangeIndex(start=10, stop=20)
index

RangeIndex(start=10, stop=20, step=1)

In [28]:
fnames[0].split('.')[0]

'TSP_Problem_0'

In [33]:
index = pd.RangeIndex(start=10, stop=20)
sol_df = pd.DataFrame()
for k, fname in enumerate(fnames):
    tmp_df = pd.DataFrame()
    tmp_df[fname.split('.')[0]] = solutions[k]
    sol_df = pd.concat([sol_df,tmp_df.astype(int)], ignore_index=False, axis=1)
    

In [36]:
sol_df.to_csv('results/{}/solution_{}.csv'.format(data_dir.split("/")[-1], best_model))

In [42]:
lens_df = pd.DataFrame()
for k, fname in enumerate(fnames):
    tmp_df = pd.DataFrame()
    tmp_df[fname.split('.')[0]] = [lengths[k]]
    lens_df = pd.concat([lens_df,tmp_df], ignore_index=False, axis=1)

In [44]:
lens_df.to_csv('results/{}/tour_lengths_{}.csv'.format(data_dir.split("/")[-1], best_model))

In [8]:
graph = graph_list[1]
length, solution, time_ = dqn.Evaluate(g=graph)

In [9]:
solution, length

([4, 6, 3, 1, 0, 8, 9, 7, 5, 2], 0.29957000000000006)

In [10]:
# check whether length calculation is correct
tot_len = 0
for i in range(np.array(solution).shape[0]):
    if i == np.array(solution).shape[0] - 1:
        tot_len += graph[solution[i]][solution[0]]['weight']
    else:
        tot_len += graph[solution[i]][solution[i + 1]]['weight']

In [11]:
tot_len/np.array(solution).shape[0]

0.29957000000000006